In [ ]:
import os

In [ ]:
import os
import tensorflow as tf
import torch
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, SimpleRNN,\
Bidirectional,SpatialDropout1D,Activation, Dropout, BatchNormalization, GlobalAveragePooling1D, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from unidecode import unidecode
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
import re
from unidecode import unidecode
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import f1_score
from numpy.random import seed
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTENC, RandomOverSampler,SVMSMOTE
from scipy.stats import rankdata

In [ ]:
train = pd.read_csv('../dados/Hackathon_Base_Treino_comdep.csv',sep=',')

In [ ]:
test = pd.read_csv('../dados/Hackathon_Base_Teste.csv',sep=',',index_col=0)

In [ ]:
dbExternoManual = pd.read_excel('../dados/baseExterno/bdManual_dotzv001.xlsx', 'Planilha5',sep=';',index_col=None)

In [ ]:
#
dbExterno = pd.read_csv('../dados/baseExterno/dbExternoV009.csv',sep=',',index_col=0)
dbExterno.columns = ['DESCRIÇÃO PARCEIRO','CATEGORIA']

In [ ]:
grupos = pd.read_csv('../envios/grupos.csv', index_col='Unnamed: 0')
from sklearn.preprocessing import LabelEncoder
cats_encoder = LabelEncoder()
grupos['categoria_encoded'] = cats_encoder.fit_transform(grupos['CATEGORIA'])

In [ ]:
train = train[['DESCRIÇÃO PARCEIRO','CATEGORIA','SUB-CATEGORIA','DEPARTAMENTO']].copy()

In [ ]:
test = test[['DESCRIÇÃO PARCEIRO']].copy()

In [ ]:
#nltk.download('stopwords')
#stopwords = nltk.corpus.stopwords.words('portuguese')


In [ ]:
REPLACE_BY_SPACE_RE = re.compile('\[/(){}\[\]\|@,;')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_-]')
STOPWORDS = set(stopwords.words('portuguese'))

In [ ]:
STOPWORDS.add('c')
STOPWORDS.add('n')
STOPWORDS.add('d')
STOPWORDS.add('+')
STOPWORDS.add('com')
STOPWORDS.add('sem')
STOPWORDS.add('para')

In [ ]:
def putCharBtw(x,letra,sep):
    rex = re.compile('[0-9]'+letra)
    if rex.search(x)==None:
        return x
    else:
        inicio = (rex.search(x).start())+1
        newString = x[:inicio]+sep+x[inicio:]
        return newString


In [ ]:
def buscaEreplace(text,term1, term2, wordreplaceTerm2):
    if term1 in text:
        if term2 in text:
            #print(text,' '+term2+' ',' '+wordreplaceTerm2+' ')
            text = text.replace(term2,wordreplaceTerm2)
            
    return text
        

In [ ]:
def changeWordBeforePosition(text,term1, term2, position):
    pos = 0
    for t in text.split():
        pos=pos+1
        if (t==term1) & (pos <= position):
            text = text.replace(term1,term2)
            #print("entrou",text)
            break
    return text

In [ ]:
def changeWordAfterPosition(text,term1, term2, position):
    pos = 0
    for t in text.split():
        pos=pos+1
        if (t==term1) & (pos <= position):
            text = text.replace(term1,term2)
            #print("entrou",text)
            break
    return text

In [ ]:
def predTop3(dtbase, predicoes):
    seq = 0
    dtbase['top1'] = 1
    dtbase['top2'] = 2
    dtbase['top3'] = 3
    
    for predicao in predicoes:        
        seq  +=1
        
        top3 = rankdata(predicao).argsort()[-3:][::-1]
        categoria1 = grupos[grupos['categoria_encoded']==top3[0]]['CATEGORIA'].max()
        categoria2 = grupos[grupos['categoria_encoded']==top3[1]]['CATEGORIA'].max()
        categoria3 = grupos[grupos['categoria_encoded']==top3[2]]['CATEGORIA'].max()

        dtbase.loc[dtbase[seq-1:seq].index,'top1'] = categoria1
        dtbase.loc[dtbase[seq-1:seq].index,'top2'] = categoria2
        dtbase.loc[dtbase[seq-1:seq].index,'top3'] = categoria3



In [ ]:
def replaceWord(text, antiga, nova):
    text = [nova if t == antiga else t for t in text.split()]
    return ' '.join(text)
    #['corte' if t in 'cortess' else t for t in text.split()]

In [ ]:
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = unidecode(text)
    text = text.replace('+', ' + ')
    text = text.replace('.', ' . ')
    text = text.replace('.', ' ')
    text = text.replace('-', ' ')
    text = text.replace('+', ' ')
    text = replaceWord(text,'d', ' ')
    text = replaceWord(text, 'n', '')
    text = replaceWord(text,'c', '')
    text = replaceWord(text, 'pcte', 'pct')
    text = replaceWord(text,'pt', 'pct')
    text = text.replace(r'c/',' com ')
    text = text.replace(r"\.c/",' com ')
    text = text.replace(r'-c/',' com ')
    text = text.replace(' s/',' sem ')
    text = text.replace(r'\.s/',' sem ')
    text = text.replace(r'-s/',' sem ')
    text = text.replace(' p/',' para ')
    text = text.replace(r'\.p/',' para ')
    text = text.replace(r'-p/',' para ')
    text = putCharBtw(text,'ml',' ')
    text = putCharBtw(text,'kg',' ')
    text = putCharBtw(text,'g',' ')
    text = putCharBtw(text,'gr',' ')
    text = putCharBtw(text,'cm',' ')
    text = putCharBtw(text,'l',' ')
    text = putCharBtw(text,'mm',' ')
    text = putCharBtw(text,'mt',' ')
    text = putCharBtw(text,'un',' ')
    text = text.replace('/',' / ')
    text = text.replace('/','')
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    #text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    text = text.replace('  ',' ')
    text = text.replace('  ',' ')
    text = replaceWord(text,'c', '')
    text = replaceWord(text,'com', '')
    text = replaceWord(text,'de', '')
    text = replaceWord(text,'sem', '')
    text = replaceWord(text,'para', '')
    text = replaceWord(text,'cortes', 'corte')
    text = replaceWord(text,'pvc', 'past')
    text = replaceWord(text,'rapadu', 'rapadura')
    text = replaceWord(text,'melanina', 'melan')
    text = replaceWord(text,'unid', 'un')
    text = text.replace('mix berrys ', 'mix grao ')
    text = text.replace(' louro cl', ' louro claro') #
    text = 'bola natal' if 'bola natal' in text else text
    text = replaceWord(text,'beefbone ', 'alim cao ')
    text = text.replace('cera capilar', 'gel fix')
    text = text.replace('cera p cab', 'gel fix')
    text = buscaEreplace(text,' 2 l', 'refriger ', 'refrigerante ')
    text = replaceWord(text,'refriger', 'refrigerador')
    text = replaceWord(text,'dental', ' dent')
    text = text.replace('humus de minhoca', 'adubo')
    text = replaceWord(text,'frutas', 'fruta')
    text = replaceWord(text,'carambola', 'carambola fruta')
    text = text.replace('cre mao ', 'creme mao ')
    text = replaceWord(text,'maos', ' mao ')
    text = replaceWord(text,'organico', 'org')
    text = replaceWord(text,'chedda', 'cheddar')
    text = replaceWord(text,'caes', 'cao')
    text = replaceWord(text,'integral', 'integ')
    text = replaceWord(text,'congelada', 'cong')
    text = replaceWord(text,'resfriado', 'resf')
    text = replaceWord(text,'resfriada', 'resf')
    text = replaceWord(text,'fresca', 'fresc')
    text = replaceWord(text,'fresco', 'fresco')
    text = replaceWord(text,'marinex', 'vidro')
    text = replaceWord(text,'esfirra', 'esfiha')
    text = text.replace('caf esfiha ', 'esfiha ')
    text = text.replace('mousse charming', 'fixador')
    text = replaceWord(text,'hamburguinho', 'hamb')
    text = replaceWord(text,'kids', 'inf')
    text = replaceWord(text,'garrafas', 'garrafa')
    text = replaceWord(text,'cuticula', 'cutic')
    text = text.replace('papel alum ', 'papel aluminio ')
    text = replaceWord(text,'champignon', 'cogumelo champignon')
    text = text.replace('cogumelo cogumelo champignon', 'cogumelo champignon')    
    text = replaceWord(text,'automotiva', 'automotivo')
    text = replaceWord(text,'polietileno', 'plast')
    text = replaceWord(text,'suin', 'suino')
    text = replaceWord(text,'suina', 'suino')
    text = replaceWord(text,'plastico', 'plast')
    text = text.replace('vinho espumante', 'espumante vinho')
    text = replaceWord(text,'festa', 'fest')
    text = replaceWord(text,'infantil', 'inf')
    text = replaceWord(text,'tabl', 'tablete')
    text = replaceWord(text,'tablet', 'tablete')
    text = replaceWord(text,'dental', 'dent')
    text = replaceWord(text,'desodorante ', 'desod')
    text = replaceWord(text,'feminino', 'fem')
    text = replaceWord(text,'multuso', 'multiuso')
    text = replaceWord(text,'m-uso', 'multiuso')
    text = replaceWord(text,'ativador ', 'ativ')
    text = replaceWord(text,'bovino ', 'bov')
    text = replaceWord(text,'baby', ' baby')
    text = replaceWord(text,'brinquedo', 'brinq')
    text = replaceWord(text,'madeira', 'mad')
    text = replaceWord(text,'hidratante', 'hidrat ')
    text = replaceWord(text,'limpeza', 'limp')
    text = replaceWord(text,'iakon', 'yakon')
    text = replaceWord(text,'pepininhos', 'pepino')
    text = replaceWord(text,'pepininho', 'pepino')
    text = replaceWord(text,'repelente', 'repel')
    text = replaceWord(text,'verde', 'vde')
    text = replaceWord(text,'bacalhau','bacal')
    text = replaceWord(text,'liquido','liq')
    text = buscaEreplace(text,'massa ', ' fr ', 'fresc ')
    text = buscaEreplace(text,'pentear', 'creme ', 'cr ')
    text = buscaEreplace(text,' dental', 'esc ', 'escova ')
    text = buscaEreplace(text,' dent', 'esc ', 'escova ')
    text = buscaEreplace(text,'escova ', 'dent', 'dental ')
    text = text.replace('ed inf', ' escova dental inf ')
    text = buscaEreplace(text,'saboneteira', 'inox ', '')
    text = buscaEreplace(text,'lixeira', 'inox ', '')
    text = buscaEreplace(text,' unha', 'inox ', '')
    text = buscaEreplace(text,'pet 2 l', 'ref ', 'refrigerante ')
    text = buscaEreplace(text,'peixe ', ' con', ' cong')
    text = replaceWord(text,'cabelos', 'cab')
    text = replaceWord(text,'cabelo', 'cab')
    text = replaceWord(text,'esfirra', 'esfiha')
    text = replaceWord(text,'esfirras', 'esfiha')
    text = replaceWord(text,'crinca', 'infantil')
    text = text.replace('sabonete ', 'sab ')
    text = replaceWord(text,'cereal', 'cereal mat')
    text = text.replace('cereal mat mat', 'cereal mat')
    text = text.replace('aroma das indias', 'temp')
    text = text.replace('bisc povilho', 'salg povilho')
    text = text.replace('biscoito de povilho', 'salg povilho')
    text = replaceWord(text,'facial', 'fac')
    text = text.replace('barras energ', 'barra cereal')
    text = replaceWord(text,'plasvale', 'plast')
    text = replaceWord(text,'sabon', 'sab')
    text = text.replace('kit cobre leito', 'cobre leito')
    text = replaceWord(text,'xicaras', 'xicara')
    text = text.replace('goma mascar', 'chiclete')
    text = replaceWord(text,'pimenta ', 'pim ')
    text = replaceWord(text,'pimen ', 'pim ')
    text = replaceWord(text,'pvc ', 'plast ')
    text = replaceWord(text,'espatulas', 'espatula')
    text = replaceWord(text,'churrasco', 'churras')
    text = text.replace('limp banheiro', 'limp banh')
    text = replaceWord(text,'termica', 'term')
    text = text.replace('pote vd', 'pote vidro')
    text = text.replace('caneca vd', 'caneca vidro')
    text = text.replace('prato fundo vd', 'prato fundo vidro')
    text = text.replace('hot chilli', 'pimenta')
    text = text.replace('sabon liq int', 'sab intimo ')
    text = text.replace('sab liq int', 'sab intimo ')
    text = text.replace('sab int', 'sab intimo ')
    text = text.replace('sabonete liquido intimo ', 'sab intimo ')
    text = text.replace('pre action', 'sup alim')
    text = replaceWord(text,'grapettinho', 'refrigerante')
    text = text.replace('ref tri cola', 'refrigerante cola ')
    text = text.replace('mist prep liq cha', 'cha ')
    text = replaceWord(text,'peix', 'peixe ')
    text = text.replace('limpador limpeza pesada', 'limp pezada ')
    text = replaceWord(text,'provolone', 'queijo provolone')
    text = text.replace('queijo queijo provolone', 'queijo provolone ')
    #text = replaceWord(text,'macarrao', 'mac')
    text = replaceWord(text,'baby', 'inf')
    text = text.replace('alimento passaro', 'alim passaro ')
    text = replaceWord(text,'panet', 'panetone')
    text = replaceWord(text,'pa higienica', 'pa hig')
    text = text.replace('file fgo', 'file frango')
    text = replaceWord(text,'masculino', 'masc')
    text = replaceWord(text,'feminino', 'fem')
    text = replaceWord(text,'corporal', 'corpo')
    text = text.replace('cr leite', 'creme de leite')
    text = replaceWord(text,'keldog', 'cao')
    text = replaceWord(text,'barbea', 'cao')
    text = replaceWord(text,'barber', 'cao')
    text = replaceWord(text,'barbear', 'cao')
    text = text.replace('cr barb', 'espuma barb')
    text = replaceWord(text,'refrigerante', 'refrig')
    text = buscaEreplace(text,'antartica', 'guarana', 'refrig guarana')
    text = text.replace('refrig refrig guarana', 'refrig guarana')
    text = replaceWord(text,'popcorn', 'pipoca')
    text = text.replace('salg coxinha', 'coxinha')
    text = text.replace('salg coxinhas', 'coxinha')
    text = replaceWord(text,'macas', 'maca')
    text = text.replace('limp lysoform', 'desinf lysoform')
    text = buscaEreplace(text,'caneca ', ' cer ', ' ceram ')
    text = text.replace('baguete com queijo', 'baguete')
    text = text.replace('baguete queijo', 'baguete')
    text = replaceWord(text,'livros', 'livro')
    text = text.replace('temp sazon', 'caldo sazon')
    text = replaceWord(text,'begonia', 'flor nat begonia')
    text = text.replace('flor nat flor nat begonia', 'flor nat begonia')
    text = replaceWord(text,'crocantissimo', 'bisc int')
    text = replaceWord(text,'carrinho', 'carro')
    text = replaceWord(text,'inseticida', 'inset')
    text = replaceWord(text,'inseti', 'inset')
    text = replaceWord(text,'sorrentinos', 'massa fr')
    text = text.replace('folha file suculento maggi', 'temp maggi')
    text = text.replace('amendoa laminada', 'amendoa casca')
    text = text.replace('barra cer ', 'barra cereal ')
    text = text.replace('supino banana', 'barra cereal supino banana')
    text = text.replace('barra cereal barra cereal supino banana', 'barra cereal supino banana')
    text = text.replace('pura nata cheddar', 'queijo cheddar pura nata')
    text = text.replace('kids kids', 'inf')
    text = text.replace('choc inf', 'choc')
    text = replaceWord(text,'goiabinha', 'goiabada ')
    text = replaceWord(text,'CarXparts', 'automotivo ')
    text = replaceWord(text,'eberg', 'energetico ')
    text = replaceWord(text,'geladinho', 'sorvete ')
    text = replaceWord(text,'quibe', 'kibe')
    text = replaceWord(text,'para', '')
    text = replaceWord(text,'legumes', 'legume')
    text = replaceWord(text,'cachorro', 'cao')
    text = replaceWord(text,'caes', 'cao')
    text = text.replace('mix leg ', 'mix legume ')
    text = text.replace('alim p cao', 'alim cao ')
    text = text.replace('biscoito cao', 'alim cao ')
    text = text.replace('coxinha da asa fgo', 'coxinha asa frango ')
    text = text.replace('coxinha asa fg', 'coxinha asa frango ')
    text = text.replace('coxinha asa', 'coxinha asa frango ')
    text = text.replace('coxinha frango cong', 'coxinha asa frango cong ')
    text = text.replace('coxinha da asa', 'coxinha asa frango ')
    text = text.replace('coxinha da asa fgo', 'coxinha asa frango ')
    text = text.replace('palito fibras', 'barra cereal ')
    text = text.replace('cozinha carnaroli', 'arroz carnaroli ')
    text = buscaEreplace(text,'abas', 'intimus ', 'abs intimus ')
    text = text.replace('abs abs intimus', 'abs intimus')
    text = replaceWord(text,'graos', 'grao')
    text = replaceWord(text,'bombocado', 'mist bolo bombocado')
    text = text.replace('mist bolo mist bolo bombocado', 'mist bolo bombocado')
    text = buscaEreplace(text,'palito', ' chur ', ' churras ')
    text = text.replace('esteira sudare', 'sushi esteira sudare')
    text = text.replace('escova profissional thermal', 'escova cab thermal')
    text = replaceWord(text,'rosbife', 'rosbife bovino')
    text = text.replace('rosbife rosbife bovino', 'rosbife bovino')
    text = replaceWord(text,'termog', 'suplem termogenico')
    text = replaceWord(text,'suplemento', 'suplem')
    text = text.replace('suplem suplem termogenico', 'suplem termogenico')
    text = replaceWord(text,'fosforos', 'fosforo')
    text = replaceWord(text,'carxparts', 'automotivo')
    text = text.replace('sab fac ', 'sab facial ')
    text = text.replace('lenc fac ', 'lenco facial ')
    text = text.replace('lenco fac ', 'lenco facial ')
    text = buscaEreplace(text,'solar', ' fac ', ' facial ')
    text = text.replace('mac ital ', 'mac ita ')
    text = text.replace('massa ital ', 'mac ita ')
    text = replaceWord(text,'organico ', 'org ')
    text = replaceWord(text,'organic ', 'org ')
    text = replaceWord(text,'organ ', 'org ')
    text = text.replace('ext virg', 'extra virgem')
    text = text.replace('beb energ', 'beb energetico')
    text = replaceWord(text,'energet ', 'energetico ')
    text = text.replace('suco nat', 'suco natural')
    text = replaceWord(text,'caderneta ', 'caderno tilibra ')
    text = buscaEreplace(text,'beijinho', 'doce ', 'doce fest beijinho ')
    text = text.replace('presilha cab', 'prendedor cab')
    text = text.replace('tilpapia ', 'tilapia ')
    text = text.replace('bisc chimanguinho', 'salgadinho chimanguinho')
    text = text.replace('bandeija ', 'bandeja ')
    text = text.replace('bandejas ', 'bandeja ')
    text = text.replace(' bandeija', ' bandeja')
    text = text.replace(' bandejas', ' bandeja')
    text = text.replace('lingua suino', 'linguica suino')
    text = text.replace('asa do brasil', '') #testar
    text = text.replace('asa do brasil', '')
    text = replaceWord(text,'caramelo', 'caram')#CARAMELO FRUITTELLA
    text = buscaEreplace(text,'fruittella', 'caram ', 'bala caram ')
    text = text.replace('fruittella ', '')
    text = buscaEreplace(text,'kg', 'barbecue', '')#M USO
    text = text.replace('m uso ', 'multiuso ')
    text = text.replace(' m uso', 'multiuso ')
    text = text.replace(' gr ', ' g ')
    text = text.replace('polpa celbola ', 'condim cebola ')
    text = text.replace('cr pentear ', 'creme pentear ')
    text = text.replace('desumidif ', 'desumidificador ')
    text = text.replace('saco vasbras', 'saco lixo')
    text = replaceWord(text,'minestrone', 'sopa legume minestrone')
    text = text.replace('salg kibe', 'kibe')
    text = replaceWord(text,'removed', 'removedor')
    text = text.replace('esmalte defense', 'defense')
    text = text.replace('esmal ', 'esmalte ')
    text = text.replace('glu tai moto', 'condim glutamato')
    text = text.replace('conj talher', 'cj talher')
    text = text.replace('gojiberry', 'goji berry')
    text = text.replace('suporte p coador', 'suporte coador')
    text = text.replace('porta filtro', 'suporte coador')
    text = replaceWord(text,'shampoo', 'sham')
    text = replaceWord(text,'bebe', 'inf')
    text = replaceWord(text,'torneiras', 'torneira')
    text = text.replace('minas frescal', 'queijo minas frescal')
    text = text.replace(' m uso ', 'multiuso')
    text = text.replace(' m uso ', 'multiuso')
    text = text.replace('tapete higienico', 'tapete hig')
    text = text.replace(' m uso ', 'multiuso')
    text = text.replace('dog', 'cao')
    text = text.replace('palinetes', 'haste flex')
    text = text.replace('hastes', 'haste')
    text = text.replace('flexiveis', 'flexivel')
    text = text.replace('haste flexivel', 'haste flex')
    text = text.replace('massa delverde', 'mac ita delverde')
    text = text.replace('triturador alho', 'amassador alho')
    text = text.replace('mist gelatina', 'gelatina po')
    text = text.replace('bolo chocolate', 'bolo choc')
    text = text.replace('cachos', 'cacho')
    text = replaceWord(text,'shopping', '')
    text = buscaEreplace(text,'power', 'gelatina', 'gelatina capilar')
    text = buscaEreplace(text,'gelatina', 'power', 'forca')
    text = buscaEreplace(text,'org ', 'batata ', 'legume batata ')
    text = text.replace('chicoria cheiro vde', 'chicoria')
    text = buscaEreplace(text,'rucula', 'vde', '')
    text = replaceWord(text,'kit', '')
    text = replaceWord(text,'condicionante', 'mascara condicionante')
    text = text.replace('mascara condicionante condicionante', 'mascara condicionante')
    text = buscaEreplace(text,'vinagre', 'vinho', '')
    text = replaceWord(text,'sacos', 'saco')
    text = replaceWord(text,'molh', 'molho')
    text = replaceWord(text,'pimen', 'pimenta')
    text = replaceWord(text,'pimentas', 'pimenta')
    text = replaceWord(text,'aliment', 'alim')
    text = replaceWord(text,'r', '')
    text = replaceWord(text,'mix', '')
    text = replaceWord(text,'piment', 'pimenta')
    text = replaceWord(text,'flores', 'flor')
    text = replaceWord(text,'silvestres', 'silvestre')
    text = replaceWord(text,'odorizador', 'purif ar')
    text = replaceWord(text,'rasteirinha', 'sandalia')
    text = replaceWord(text,'filhotes', 'filhote')
    text = replaceWord(text,'gatos', 'gato')
    text = replaceWord(text,'mamadeiras', 'mamadeira')
    text = replaceWord(text,'porco', 'suino')
    text = replaceWord(text,'bag', '')
    text = replaceWord(text,'animais', 'animal')
    text = replaceWord(text,'cuticulas', 'cuticula')
    
    text = text.replace('ferro passar', 'ferro vapor')
    text = text.replace('massa peixe tikuwa', 'massa tikuwa')
    text = text.replace('forno magico', 'forno magico')
    text = text.replace('pe moca', 'pe moleque ')
    text = text.replace('pe moca', 'pe moleque ')
    text = text.replace('carro boneca', 'boneca ')
    text = text.replace('carros', 'carro ')
    text = text.replace('bonecos', 'boneco ')
    text = text.replace('coco sec', 'coco ')
    text = text.replace('gel acendedor', 'churras ')
    text = text.replace('auto ', 'auto ')
    text = text.replace('leite pentear ', 'cr pentear ')
    text = text.replace('cre queijo', 'creme queijo ')
    text = text.replace('cr queijo', 'creme pentear ')
    text = text.replace('pres qjo', 'presunto queijo ')
    text = text.replace('pastilha sanit ', 'desod sanit ')
    text = text.replace('caldo sazon', 'temp sazon')
    text = text.replace('caldo sazon', 'temp sazon')
    text = text.replace('sha cond cao', 'shamp cond cao')
    text = text.replace('limpz ', 'limp ')
    text = text.replace('hot wings ', 'asa frango ')
    text = text.replace('balalaika ', 'vodka ')
    text = text.replace('queijo ricota ', 'ricota ')
    text = text.replace('fresco cong ', 'fresco ') #
    text = text.replace('bloco odorizante ', 'odorizante ')
    text = text.replace('figo seco ', 'fruta figo ')
    text = text.replace('alicate cut', 'alicate cuticula unha')
    
    
    
    text = buscaEreplace(text,'pinho', 'pastilha ', 'desod sanit')    
    text = buscaEreplace(text,'presunto', 'rondelli', 'sanduiche rondelli')
    text = buscaEreplace(text,'pres ', 'sand', 'sanduiche')
    text = buscaEreplace(text,'sanduiche ', 'pres', 'presunto')
    text = buscaEreplace(text,'escova ', 'banh ', 'banho ')
    text = buscaEreplace(text,'margarina ', 'cap ', ' ')
    text = buscaEreplace(text,'bolo ', 'leite cond', ' ')
    text = buscaEreplace(text,'bolo ', 'leite cond', ' ')
    text = buscaEreplace(text,'pimenta ', 'biq ', 'biquinho ')
    text = buscaEreplace(text,'vela', 'pote vidro', '')
    text = buscaEreplace(text,'nectar', 'mel ', 'mel silvestre')
    text = buscaEreplace(text,'limo', 'limp banh', '')
    text = buscaEreplace(text,'vodka', 'beb alc', '')
    text = buscaEreplace(text,'suino', 'fresco', 'resf')#fresco cong
    text = buscaEreplace(text,'pom pom', 'sab', 'sab inf')
    
    
    text = text.replace('  ',' ')
    text = text.replace('  ',' ')
    text = text.replace('  ',' ')
    #    text = re.sub(r'\W+', '', text)
    return text

In [ ]:
dadosFull = pd.concat([train[['DESCRIÇÃO PARCEIRO','CATEGORIA','SUB-CATEGORIA']],
                       dbExterno[['DESCRIÇÃO PARCEIRO','CATEGORIA']],
                       dbExternoManual[['DESCRIÇÃO PARCEIRO','CATEGORIA']],
                       test[['DESCRIÇÃO PARCEIRO']]], ignore_index=True, sort=False)
dadosFull['descricao'] = dadosFull['DESCRIÇÃO PARCEIRO'].apply(clean_text)

In [ ]:
#dadosFull[(dadosFull['descricao'].str.contains('milho vde'))]['descricao']

In [ ]:
dadosFull[(dadosFull['descricao'].str.contains('hortela'))].head()

In [ ]:
dadosFull[ (dadosFull['CATEGORIA']=='DOCE FESTA')].head()

In [ ]:
#tokenizerDotz.texts_to_sequences("ed inf oral b stages 1 4 24 meses")


In [ ]:
#dadosFull[11131:11132]['descricao'].values

In [ ]:
list = [13504,21484,21441,19480,693,9397,7166,5366,18948,10286,10286,11435,18623,12209,
        8809,13279,2484,6578,90,5391,12674,18513,8744,13282,9653,3476,8815,
       8815, 21920, 17272,541,2678,5822,17175,4086,13383,10271,19019,13383,
       20943,2290,20943,20233,1804,11322,14309,11322,8181,1676,15275,
       13665,12618,18697,9249,4467,12164]
pd.options.display.max_rows = 200
#dadosFull.loc[list].head(73)

In [ ]:
dadosFull.drop([13504,21484,21441,19480,693,9397,7166,5366,18948,10286,10286,11435,18623,12209,
        8809,13279,2484,6578,90,5391,12674,18513,8744,13282,9653,3476,8815,
       8815, 21920, 17272,541,2678,5822,17175,4086,13383,10271,19019,13383,
       20943,2290,20943,20233,1804,11322,14309,11322,8181,1676,15275,
       13665,12618,18697,9249,4467,12164], axis=0, inplace=True)

dadosFull.reset_index(drop=True)

In [ ]:
tokenizerDotz = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True,split=' ') 
tokenizerDotz.fit_on_texts(dadosFull['descricao'].values)
word_index = tokenizerDotz.word_index 
print('Found %s unique tokens.' % len(word_index))
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = len(word_index)+1

In [ ]:
trainX = dadosFull[~dadosFull['CATEGORIA'].isnull()].copy()
#trainX = trainX[:31442].copy()
#X_test1ext = dadosFull[31442:].copy()
trainY = dadosFull[~dadosFull['CATEGORIA'].isnull()]['CATEGORIA']
#trainY = trainY[:31442].copy()
#Y_test1ext = dadosFull[31442:].copy()

In [ ]:
#trainAll = trainX.copy()
#trainAll['CATEGORIA'] = trainY

In [ ]:
#cattrain_encoder = LabelEncoder()
#trainAll['subcat_encode'] = cats_encoder.fit_transform(trainAll['CATEGORIA'])
#categorias = trainY.value_counts()
#reslist = []
#for index, qtdreg in categorias.iteritems():
#    if qtdreg < 100:
#        reslist.append(100)
#    else: 
#        reslist.append(qtdreg)

In [ ]:
#trainAll['CATEGORIA'].shape 

In [ ]:
"""
reslist = {}
for index, subcat_encode in trainAll['subcat_encode'].iteritems():
    categoria = trainAll.loc[index]['CATEGORIA']
    reslist[categoria] = \
    20 if trainAll[(trainAll['CATEGORIA']==categoria)]['CATEGORIA'].count() < 20 \
    else trainAll[(trainAll['CATEGORIA']==categoria)]['CATEGORIA'].count() 
ros = RandomOverSampler(random_state=42, sampling_strategy=reslist)
trainX, trainY = ros.fit_resample(trainAll, 
                                  trainAll[['CATEGORIA']])#sampling_strategy= 'minority'SVMSMOTE
"""

In [ ]:
#sampling_strategy= 'minority'SVMSMOTE
max([len(x) for x in trainX])

In [ ]:
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(trainX,trainY, 
                                                        test_size = 0.0000001, 
                                                        random_state = 1215)##54,154,stratify=trainY
print(X_train1.shape,Y_train1.shape) 
print(X_test1.shape,Y_test1.shape)

In [ ]:
X_train = X_train1[~X_train1['CATEGORIA'].isnull()]['descricao']
X_test = X_test1[~X_test1['CATEGORIA'].isnull()]['descricao']
#X_test1ext = X_test1ext[~X_test1ext['CATEGORIA'].isnull()]['descricao']

In [ ]:
X_train = tokenizerDotz.texts_to_sequences(X_train.values)
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = max([len(x) for x in trainX])
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
print('Shape of data tensor:', X_train.shape, 'MAX_SEQUENCE_LENGTH', MAX_SEQUENCE_LENGTH)

In [ ]:
X_test = tokenizerDotz.texts_to_sequences(X_test.values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
print('Shape of data tensor:', X_test.shape)

#X_test1ext = tokenizerDotz.texts_to_sequences(X_test1ext.values)
#X_test1ext = pad_sequences(X_test1ext, maxlen=MAX_SEQUENCE_LENGTH)
#print('Shape of data tensor:', X_test1ext.shape)

In [ ]:
Y_train1 = pd.get_dummies(Y_train1).values
Y_test1 = pd.get_dummies(Y_test1).values
print('Shape of label tensor:', Y_train1.shape)
print('Shape of label tensor:', Y_test1.shape)

In [ ]:
#ros = SVMSMOTE(random_state=42, sampling_strategy='minority')
#X_res, y_res = ros.fit_resample(X_train, Y_train1)

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(trainY),
                                                 trainY)
class_weights = dict(enumerate(class_weights))

In [ ]:
opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
#lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#    initial_learning_rate=1e-2,
#    decay_steps=10000,
#    decay_rate=0.9)
# opt = keras.optimizers.Adam(learning_rate=0.01)
# This is fixed.
EMBEDDING_DIM = 322
batch_size = 64
#seed_value = 14
#seed(seed_value)
#tf.random.set_seed(seed_value)
#np.random.seed(seed_value)
#tf.set_seed = seed_value
#os.environ['PYTHONHASHSEED']=str(seed_value)

#with tf.device('/CPU:0'):
#with tf.device('/CPU:0'):
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
#model.add(Bidirectional(LSTM(322)))#,dropout=0.2,return_sequences=True
#model.add(Bidirectional(LSTM(322)))
model.add(GRU(322))#
#model.add(SimpleRNN(512))
#model.add(GlobalAveragePooling1D())
#model.add(Activation('relu'))
#model.add(Dropout(0.1,seed=seed_value))
model.add(Dense(322))#Y_train1.shape[1]
model.add(BatchNormalization(axis=1, momentum=0.8)) #axis=1, momentum=0.08
#model.add(Dropout(0.2))
#model.add(Activation('relu'))

model.add(Dense(Y_train1.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())    

In [ ]:
epochs = 5
history = model.fit(X_train, Y_train1, epochs=epochs, 
                    batch_size=batch_size,validation_split=0.07,#class_weight=class_weights,# shuffle=False,validation_batch_size=batch_size,
                    callbacks=[EarlyStopping(monitor='accuracy', patience=3, 
                                             min_delta=0.0001,restore_best_weights=True)])

In [ ]:
epochs = 2
history = model.fit(X_train, Y_train1, epochs=epochs, 
                    batch_size=batch_size,validation_split=0.08,#class_weight=class_weights,# shuffle=False,validation_batch_size=batch_size,
                    callbacks=[EarlyStopping(monitor='accuracy', patience=1, 
                                             min_delta=0.0001,restore_best_weights=True)])

# ANALISE DE RESULTADOS

In [ ]:
model.save('../modelos/DotzV017-CategoriaV015-2331')

In [ ]:
#model = tf.keras.models.load_model('../modelos/DotzV017-tensorFlowNLPMulticlass-CategoriaV010')

# GERACAO DO ENVIO

In [ ]:
test['descricao'] = test['DESCRIÇÃO PARCEIRO'].apply(clean_text)

In [ ]:
seq = tokenizerDotz.texts_to_sequences(test['descricao'].values)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
predicoes = model.predict(padded)
predicted = np.argmax(model.predict(padded), axis=-1)

In [ ]:
testEnvio = test.copy()

In [ ]:
testEnvio['categoria_encoded'] = predicted

In [ ]:
predTop3(testEnvio,predicoes)

In [ ]:
testEnvio

In [ ]:
testEnvio.to_csv('../envios/preenvio/cat_envioTesteV042_2331.csv', index_label='0')

# END